In [1]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\usvidr03\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\usvidr03\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [3]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"PSI - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [4]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [5]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyMuPDFLoader


path = "bills/"
loader = DirectoryLoader(path, glob="*.pdf", loader_cls=PyMuPDFLoader)
docs = loader.load()

In [6]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

In [7]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

In [8]:
from ragas.testset.graph import Node, NodeType

### NOTICE: We're using a subset of the data for this example - this is to keep costs/time down.
for doc in docs[:20]:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 20, relationships: 0)

In [9]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying SummaryExtractor:   0%|          | 0/16 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/20 [00:00<?, ?it/s]

Node 19611763-d813-42f5-9c52-86744189cefb does not have a summary. Skipping filtering.
Node 7d127c0a-0078-456b-8786-f2ba75a3b936 does not have a summary. Skipping filtering.
Node 64cdf325-784d-4949-af68-3c5a22defc41 does not have a summary. Skipping filtering.
Node 61933dd5-7b14-4c2d-8f16-184124746249 does not have a summary. Skipping filtering.


Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/56 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 20, relationships: 133)

In [10]:
kg.save("bills/ai_law.json")
bills_data_kg = KnowledgeGraph.load("bills/ai_law.json")
bills_data_kg

KnowledgeGraph(nodes: 20, relationships: 133)

In [11]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=bills_data_kg)

In [12]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

Finally, we can use our `TestSetGenerator` to generate our testset!

In [13]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,What is the significance of CD0 Magazine in th...,[TWENTIETH CONGRESS OF THE \nREPUBLIC OF THE P...,The context mentions CD0 Magazine as the sourc...,single_hop_specifc_query_synthesizer
1,What are the concerns associated with Artifici...,[AI presents enormous opportunities for the Ph...,The concerns regarding ASI include the possibi...,single_hop_specifc_query_synthesizer
2,"What is the significance of Articel XIV, Secio...",[TWENTIETH CONGRESS OF THE \nREPUBLIC OF THE P...,"The context mentions Article XIV, Section 10 o...",single_hop_specifc_query_synthesizer
3,Who are the Filipinos?,"[1 \na) Promote innovation, technological adva...",The context mentions Filipinos as the group wh...,single_hop_specifc_query_synthesizer
4,What are the implications of Artificial Superi...,[1\n2\n3\n4\n5\n6\n7\n8 \n9\n10\n11\n12\n13\n1...,Artificial Superintelligence (ASI) refers to h...,single_hop_specifc_query_synthesizer
5,How does the law's effective date relate to su...,"[<1-hop>\n\n1 \nendorsements, voice recordings...",The context indicates that the law shall take ...,multi_hop_abstract_query_synthesizer
6,How does the framework for responsible AI deve...,[<1-hop>\n\nAI presents enormous opportunities...,The context explains that the bill aims to bal...,multi_hop_abstract_query_synthesizer
7,how do violations of safegards and skills mapp...,[<1-hop>\n\n1 \ndevelopment priorities. These ...,"Violations of safeguards, rules, and regulatio...",multi_hop_abstract_query_synthesizer
8,How does the AI Ethics Review Board's role in ...,[<1-hop>\n\n1 \ndevelopment priorities. These ...,The AI Ethics Review Board is responsible for ...,multi_hop_specific_query_synthesizer
9,How does the Philippine DICT's role in establi...,[<1-hop>\n\n1 \nSec. 6. Jurisdiction of the NA...,The law assigns the DICT the role of supportin...,multi_hop_specific_query_synthesizer


In [15]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs[:10], testset_size=5)

Applying SummaryExtractor:   0%|          | 0/10 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/10 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/30 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/6 [00:00<?, ?it/s]

In [16]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,What is the significance of the REPUBLIC OF TH...,[TWENTIETH CONGRESS OF THE \nREPUBLIC OF THE P...,The context discusses the TWENTIETH CONGRESS O...,single_hop_specifc_query_synthesizer
1,What is ASI and why is it like a big deal for ...,[AI presents enormous opportunities for the Ph...,The context mentions that the global scientifi...,single_hop_specifc_query_synthesizer
2,How does the Philippines' development of a mul...,[<1-hop>\n\n1 \nk) Issue advisory opinions on ...,The Philippines' development of a multi-year A...,multi_hop_abstract_query_synthesizer
3,how do they do certification and monitoring of...,[<1-hop>\n\n1 \nSec. 8. NAICSecretariat. - The...,The law states that the NAIC shall certify and...,multi_hop_abstract_query_synthesizer
4,How does the development of Artificial Superin...,[<1-hop>\n\nAI presents enormous opportunities...,The context highlights that AI presents signif...,multi_hop_specific_query_synthesizer
5,How does the DOST relate to the NAIC's jurisdi...,[<1-hop>\n\n1 \nSec. 6. Jurisdiction of the NA...,"The NAIC, which is attached to the Department ...",multi_hop_specific_query_synthesizer


In [29]:
import pandas as pd
df = dataset.to_pandas()
pd.set_option('display.max_colwidth', None)
pd.set_option("display.colheader_justify", "left")

print(df['user_input'].to_string())

0                                                                                                                                                                                                                                                                                                                                                                                                                                  What is the significance of the REPUBLIC OF THE PHILIPPINES in the context of AI regulation and development?
1                                                                                                                                                                                                                                                                                                                                                                                                                       What is ASI and why is it like a big deal for AI research and de

We'll need to provide our LangSmith API key, and set tracing to "true".

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [31]:
from langsmith import Client

client = Client()

dataset_name = "PH_AI_BILL_2025"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Philippines AI Bills 2025"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [32]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

In [33]:
rag_documents = docs

In [34]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

In [35]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [36]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="AI Bills RAG"
)

In [37]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

In [38]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

For our LLM, we will be using TogetherAI's endpoints as well!

We're going to be using Meta Llama 3.1 70B Instruct Turbo - a powerful model which should get us powerful results!

In [39]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")

Finally, we can set-up our RAG LCEL chain!

In [40]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [41]:
rag_chain.invoke({"question" : "How much is the penalty for spreading disinformation?"})

'The penalty for using AI to create or disseminate disinformation is a fine of One Million Pesos (Php 1,000,000) to Five Million Pesos (Php 5,000,000), or imprisonment of three (3) years to ten (10) years, or both, at the discretion of the court.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4.1 as our evaluation LLM for our base Evaluators.

In [42]:
eval_llm = ChatOpenAI(model="gpt-4.1")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [43]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

empathy_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "empathy": "Is this response empathetic? Does it make the user feel like they are being heard?",
        },
        "llm" : eval_llm
    }
)

## LangSmith Evaluation

Basic Rag Chain

In [44]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'whispered-lettuce-34' at:
https://smith.langchain.com/o/73c7812f-fb16-4468-bb8f-115ba901532a/datasets/21fd1cce-754b-4695-81f3-4f8aefa58a02/compare?selectedSessions=9533ed6c-a6ea-4655-9f55-4163b257200b




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,How does the DOST relate to the NAIC's jurisdiction over AI development and regulation?,"Based on the provided context, the NAIC (National AI Commission) is an agency attached to the Department of Science and Technology (DOST). The NAIC has original and exclusive jurisdiction over all matters pertaining to AI, including its development, promotion, registration, and regulation. The Secretary of the DOST also serves as the Chairman of the NAIC.\n\nTherefore, the DOST relates to the NAIC's jurisdiction over AI development and regulation by being the parent department to which the NAIC is attached, and through the Secretary of the DOST who chairs the NAIC, providing leadership and oversight in these matters.",None,"The NAIC, which is attached to the Department of Science and Technology (DOST), has exclusive jurisdiction over all matters pertaining to AI, including its development, promotion, registration, and regulation. The NAIC is responsible for technical support and policy alignment across government agencies involved in AI, and it can impose administrative penalties for violations, establishing a direct relationship between DOST and the NAIC's AI oversight functions.",1,1,0,2.450819,5b0bdfed-5b90-4119-82a9-a30f2ac69a3f,1f5d6877-d2f1-4c8f-9125-e6604b9565fa
1,"How does the development of Artificial Superintelligence (ASI) in the Philippines, as discussed in the context, relate to the global concerns about ASI surpassing human control and the need for responsible regulation to ensure safety and ethical use?","The development of Artificial Superintelligence (ASI) in the Philippines is viewed within the context of global concerns about ASI surpassing human intelligence and potentially going beyond human control. The context highlights alarms raised by the global scientific and policy communities regarding these risks, which include the possibility that improperly secured or regulated ASI systems might cause harm.\n\nAccordingly, the Philippines recognizes the urgent need for a national framework to ensure the safe, responsible, and ethical use of AI, including ASI. This framework aims to prevent AI from being used to perpetrate crimes, abuse rights, or cause harm—whether intentional or accidental. The State bears the responsibility to regulate AI to support Filipino ingenuity and technological progress while addressing these risks.\n\nThus, the development of ASI in the Philippines is closely related to global concerns by emphasizing responsible regulation and oversight to manage potential risks and ensure AI technologies are aligned with ethical standards and public safety.",None,"The context highlights that AI presents significant opportunities for the Philippines, including improving public services and advancing technological development. However, it also raises concerns about the rise of Artificial Superintelligence (ASI), a hypothetical AI system surpassing human intelligence and potentially beyond human control. Globally, scientific and policy communities have expressed alarms regarding the risks of ASI gaining unauthorized access to critical infrastructure and posing existential threats. The Philippines' approach, as outlined, emphasizes responsible development and regulation of AI, including ASI, to balance innovation with safety, ethics, and human oversight. This aligns with international concerns by advocating for policies that ensure ASI remains secure, transparent, and under meaningful human control to prevent misuse or unintended consequences.",1,1,0,3.845638,cec2f7f8-bf02-482c-b1d6-484da9ee5a5f,f94fbcbd-be98-4985-9db5-d40fda209e11
2,"how do they do certification and monitoring of AI risks like in the law, and how does that relate to certification, and what does it mean for AI safety and risks, and how is it done in the law, and what are the rules for certifying AI risks, an

In [47]:
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI(model="gpt-4.1-nano")

In [45]:
from langchain_community.retrievers import BM25Retriever

BM25 Retriever

In [50]:
# ----------------------
bm25_retriever = BM25Retriever.from_documents(rag_documents)
bm25_retriever.k = 10   # same effect as search_kwargs={"k": 10}

bm25_retrieval_chain = (
    {"context": itemgetter("question") | bm25_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | rag_prompt | llm | StrOutputParser()
)

In [51]:
evaluate(
    bm25_retrieval_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "chain_init_bm25"},
)



View the evaluation results for experiment: 'proper-weather-9' at:
https://smith.langchain.com/o/73c7812f-fb16-4468-bb8f-115ba901532a/datasets/21fd1cce-754b-4695-81f3-4f8aefa58a02/compare?selectedSessions=351981c8-5401-45ac-b0ab-7c6ef43fbdaf




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,How does the DOST relate to the NAIC's jurisdiction over AI development and regulation?,"Based on the provided context, the DOST (Department of Science and Technology) is connected to the NAIC (presumably the National AI Commission) in several ways related to AI development and regulation:\n\n- The NAIC has original and exclusive jurisdiction over all matters pertaining to AI, including its development, promotion, registration, and regulation (Sec. 6).\n- The NAIC includes, among its members, the Secretary of the DOST as Chairman (Sec. 7).\n- The NAIC is responsible for technical support and policy alignment of all government offices involved in AI development, including those under or attached to the DOST and other concerned agencies (Sec. 6).\n- The NAIC has the authority to impose administrative penalties for violations of the Act (Sec. 6).\n \nTherefore, the DOST has a leadership role within the NAIC (its Secretary being the Chairman) and works closely through the NAIC to align policies and provide technical support for AI development and regulation across government offices, including those attached to the DOST.\n\n# Final answer\n\nThe DOST is integrally involved in the NAIC’s jurisdiction over AI as its Secretary serves as the NAIC Chairman, and the NAIC aligns technical support and policies across government offices, including those under or attached to the DOST, thus ensuring DOST’s leadership and active role in AI development and regulation.",None,"The NAIC, which is attached to the Department of Science and Technology (DOST), has exclusive jurisdiction over all matters pertaining to AI, including its development, promotion, registration, and regulation. The NAIC is responsible for technical support and policy alignment across government agencies involved in AI, and it can impose administrative penalties for violations, establishing a direct relationship between DOST and the NAIC's AI oversight functions.",1,1,0,5.325980,5b0bdfed-5b90-4119-82a9-a30f2ac69a3f,3a3e8df3-e175-46df-99f4-c94b1b915e82
1,"How does the development of Artificial Superintelligence (ASI) in the Philippines, as discussed in the context, relate to the global concerns about ASI surpassing human control and the need for responsible regulation to ensure safety and ethical use?","The context provided indicates that global scientific and policy communities have raised alarms regarding the possible rise of Artificial Superintelligence (ASI), which refers to hypothetical AI systems surpassing human intelligence and potentially going beyond human control. This risk includes fears that such systems, if improperly secured or regulated, might gain unchecked power.\n\nIn the Philippines, the development of AI, including concerns about ASI, is acknowledged within efforts to establish a clear legal and regulatory framework aimed at protecting the rights of Filipinos and promoting responsible AI adoption and innovation. The measures discussed emphasize the importance of effective regulation, programmatic direction, balancing regulation with consumer protection, and utilizing approaches like regulatory sandboxes to foster innovation while managing risks.\n\nTherefore, the development of ASI in the Philippines aligns with global concerns by recognizing the potential risks of ASI surpassing human control and underscores the necessity for responsible regulation to ensure safety, ethical use, and to address social inequities. The Philippines aims to address these challenges through legal and regulatory frameworks modeled to promote transparency, accountability, ethics, and consumer protection in AI deployment.\n\nIn summary, the Philippine approach reflects global concerns about ASI by emphasizing responsible regulation and safeguards to ensure ASI development is controlled, ethical, and beneficial to society.",None,"The cont

Multi Query Retriever

In [54]:
from langchain.retrievers.multi_query import MultiQueryRetriever

multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=retriever, llm=llm
)

In [55]:
multi_query_retrieval_chain = (
    {"context": itemgetter("question") | multi_query_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | rag_prompt | llm | StrOutputParser()
)

In [56]:
evaluate(
    multi_query_retrieval_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "chain_init_multi_query"},
)

View the evaluation results for experiment: 'vacant-match-22' at:
https://smith.langchain.com/o/73c7812f-fb16-4468-bb8f-115ba901532a/datasets/21fd1cce-754b-4695-81f3-4f8aefa58a02/compare?selectedSessions=fe1ab4d6-b0ad-4dbc-8d37-e137c910f12f




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,How does the DOST relate to the NAIC's jurisdiction over AI development and regulation?,"Based on the provided context, the National AI Commission (NAIC) is an agency attached to the Department of Science and Technology (DOST). The NAIC has original and exclusive jurisdiction over all matters pertaining to AI, including its development, promotion, registration, and regulation. Additionally, the Secretary of the DOST serves as the Chairman of the NAIC. Therefore, the DOST is directly connected to the NAIC's jurisdiction through its administrative attachment and leadership role, supporting and overseeing the NAIC's authority over AI development and regulation.",None,"The NAIC, which is attached to the Department of Science and Technology (DOST), has exclusive jurisdiction over all matters pertaining to AI, including its development, promotion, registration, and regulation. The NAIC is responsible for technical support and policy alignment across government agencies involved in AI, and it can impose administrative penalties for violations, establishing a direct relationship between DOST and the NAIC's AI oversight functions.",1,1,0,5.076947,5b0bdfed-5b90-4119-82a9-a30f2ac69a3f,f3e697ee-308c-427b-be26-53c6e2bacfb7
1,"How does the development of Artificial Superintelligence (ASI) in the Philippines, as discussed in the context, relate to the global concerns about ASI surpassing human control and the need for responsible regulation to ensure safety and ethical use?","The development of Artificial Superintelligence (ASI) in the Philippines is recognized within the context of global concerns that such AI systems could surpass human intelligence and potentially go beyond human control. This risk includes fears that improperly secured or regulated ASI might gain harmful autonomy. In response, the context highlights the urgent need for a national framework in the Philippines to ensure the safe, responsible, and ethical use of AI. This framework aims to protect the rights and welfare of citizens, prevent AI from perpetrating crimes or causing harm (whether intentional or accidental), and support technological progress in a responsible manner. Thus, the development of ASI in the Philippines is closely tied to these global concerns, emphasizing the necessity for strong regulation and oversight to manage AI risks and align AI advancements with national safety and ethical standards.",None,"The context highlights that AI presents significant opportunities for the Philippines, including improving public services and advancing technological development. However, it also raises concerns about the rise of Artificial Superintelligence (ASI), a hypothetical AI system surpassing human intelligence and potentially beyond human control. Globally, scientific and policy communities have expressed alarms regarding the risks of ASI gaining unauthorized access to critical infrastructure and posing existential threats. The Philippines' approach, as outlined, emphasizes responsible development and regulation of AI, including ASI, to balance innovation with safety, ethics, and human oversight. This aligns with international concerns by advocating for policies that ensure ASI remains secure, transparent, and under meaningful human control to prevent misuse or unintended consequences.",1,1,0,8.231049,cec2f7f8-bf02-482c-b1d6-484da9ee5a5f,35b44c3a-d828-47af-80cf-788b6efd5dce
2,"how do they do certification and monitoring of AI risks like in the law, and how does that relate to certification, and what does it mean for AI safety and risks, and how is it done in the law, and what are the rules for certifying AI risks, and how does that help with AI safety and risks, and what are the rules for certifying AI risks in the law, and how do they do it, and what are the rules for certifying AI risks, and how does that help AI safet

Parent Document Retriever

In [57]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from qdrant_client import QdrantClient, models

parent_docs = rag_documents
child_splitter = RecursiveCharacterTextSplitter(chunk_size=750)

In [58]:
from langchain_qdrant import QdrantVectorStore

client = QdrantClient(location=":memory:")

client.create_collection(
    collection_name="full_documents",
    vectors_config=models.VectorParams(size=1536, distance=models.Distance.COSINE)
)

parent_document_vectorstore = QdrantVectorStore(
    collection_name="full_documents", embedding=OpenAIEmbeddings(model="text-embedding-3-small"), client=client
)

In [59]:
store = InMemoryStore()

parent_document_retriever = ParentDocumentRetriever(
    vectorstore = parent_document_vectorstore,
    docstore=store,
    child_splitter=child_splitter,
)

In [60]:
parent_document_retriever.add_documents(parent_docs, ids=None)

In [61]:
parent_document_retrieval_chain = (
    {"context": itemgetter("question") | parent_document_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | rag_prompt | llm | StrOutputParser()
)

In [62]:
evaluate(
    parent_document_retrieval_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "chain_init_parent_doc"},
)

View the evaluation results for experiment: 'notable-seat-51' at:
https://smith.langchain.com/o/73c7812f-fb16-4468-bb8f-115ba901532a/datasets/21fd1cce-754b-4695-81f3-4f8aefa58a02/compare?selectedSessions=33ee55fe-4105-40b9-8c9d-ede67fe769b9




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,How does the DOST relate to the NAIC's jurisdiction over AI development and regulation?,"Based on the context, the DOST is directly involved in the NAIC's jurisdiction over AI development and regulation. The NAIC, which has original and exclusive jurisdiction over all matters pertaining to AI—including its development, promotion, registration, and regulation—includes the Secretary of the DOST as its Chairman. Additionally, the NAIC is responsible for technical support and policy alignment related to AI, involving government offices under or attached to the DOST and other concerned agencies involved in AI development. Therefore, the DOST plays a leading role within the NAIC in overseeing and supporting AI regulation and development activities.",None,"The NAIC, which is attached to the Department of Science and Technology (DOST), has exclusive jurisdiction over all matters pertaining to AI, including its development, promotion, registration, and regulation. The NAIC is responsible for technical support and policy alignment across government agencies involved in AI, and it can impose administrative penalties for violations, establishing a direct relationship between DOST and the NAIC's AI oversight functions.",1,1,0,4.318922,5b0bdfed-5b90-4119-82a9-a30f2ac69a3f,aee372a1-3e5a-4910-84f4-88ada6f6d8b1
1,"How does the development of Artificial Superintelligence (ASI) in the Philippines, as discussed in the context, relate to the global concerns about ASI surpassing human control and the need for responsible regulation to ensure safety and ethical use?",I don't know.,None,"The context highlights that AI presents significant opportunities for the Philippines, including improving public services and advancing technological development. However, it also raises concerns about the rise of Artificial Superintelligence (ASI), a hypothetical AI system surpassing human intelligence and potentially beyond human control. Globally, scientific and policy communities have expressed alarms regarding the risks of ASI gaining unauthorized access to critical infrastructure and posing existential threats. The Philippines' approach, as outlined, emphasizes responsible development and regulation of AI, including ASI, to balance innovation with safety, ethics, and human oversight. This aligns with international concerns by advocating for policies that ensure ASI remains secure, transparent, and under meaningful human control to prevent misuse or unintended consequences.",0,0,0,1.160848,cec2f7f8-bf02-482c-b1d6-484da9ee5a5f,10687101-850d-4184-92e2-9f5de7365574
2,"how do they do certification and monitoring of AI risks like in the law, and how does that relate to certification, and what does it mean for AI safety and risks, and how is it done in the law, and what are the rules for certifying AI risks, and how does that help with AI safety and risks, and what are the rules for certifying AI risks in the law, and how do they do it, and what are the rules for certifying AI risks, and how does that help AI safety and risks, and what does certification mean for AI risks and safety?","Based on the provided context from the law:\n\n1. **Certification and Monitoring of AI Risks:**\n - AI systems classified as high-risk must undergo **mandatory algorithmic impact assessments, data privacy reviews, and sustainability screening** before they can be certified for use.\n - This certification process ensures that high-risk AI applications are thoroughly evaluated for potential harms and compliance with ethical and legal standards.\n \n2. **Relation to AI Safety and Risks:**\n - By subjecting high-risk AI systems to these rigorous assessments and reviews before certification, the law aims to identify and mitigate risks proactively.\n - This process helps to minimize potential harm to individuals, organizations, or social systems, th

Compression Retriever

In [64]:
os.environ["COHERE_API_KEY"] = getpass.getpass("Cohere API Key:")

In [65]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

compressor = CohereRerank(model="rerank-v3.5")
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

In [66]:
contextual_compression_retrieval_chain = (
    {"context": itemgetter("question") | compression_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | rag_prompt | llm | StrOutputParser()
)

In [67]:
evaluate(
    contextual_compression_retrieval_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "chain_init_compression"},
)

View the evaluation results for experiment: 'scholarly-society-6' at:
https://smith.langchain.com/o/73c7812f-fb16-4468-bb8f-115ba901532a/datasets/21fd1cce-754b-4695-81f3-4f8aefa58a02/compare?selectedSessions=3ac99ac9-c611-4754-a24b-d525965fff43




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,How does the DOST relate to the NAIC's jurisdiction over AI development and regulation?,"Based on the context, the NAIC (National AI Commission) is an agency attached to the Department of Science and Technology (DOST). The NAIC has the original and exclusive jurisdiction over all matters pertaining to AI, including its development, promotion, registration, and regulation. Therefore, the DOST relates to the NAIC by being the parent agency to which the NAIC is attached, while the NAIC holds the jurisdiction over AI matters.",None,"The NAIC, which is attached to the Department of Science and Technology (DOST), has exclusive jurisdiction over all matters pertaining to AI, including its development, promotion, registration, and regulation. The NAIC is responsible for technical support and policy alignment across government agencies involved in AI, and it can impose administrative penalties for violations, establishing a direct relationship between DOST and the NAIC's AI oversight functions.",1,1,0,3.611072,5b0bdfed-5b90-4119-82a9-a30f2ac69a3f,6a9fd6d9-590c-4c1a-8f58-6a8d9e1633c8
1,"How does the development of Artificial Superintelligence (ASI) in the Philippines, as discussed in the context, relate to the global concerns about ASI surpassing human control and the need for responsible regulation to ensure safety and ethical use?",I don't know.,None,"The context highlights that AI presents significant opportunities for the Philippines, including improving public services and advancing technological development. However, it also raises concerns about the rise of Artificial Superintelligence (ASI), a hypothetical AI system surpassing human intelligence and potentially beyond human control. Globally, scientific and policy communities have expressed alarms regarding the risks of ASI gaining unauthorized access to critical infrastructure and posing existential threats. The Philippines' approach, as outlined, emphasizes responsible development and regulation of AI, including ASI, to balance innovation with safety, ethics, and human oversight. This aligns with international concerns by advocating for policies that ensure ASI remains secure, transparent, and under meaningful human control to prevent misuse or unintended consequences.",0,0,0,1.438988,cec2f7f8-bf02-482c-b1d6-484da9ee5a5f,65e313e0-87a3-4c40-90bd-5d1472631724
2,"how do they do certification and monitoring of AI risks like in the law, and how does that relate to certification, and what does it mean for AI safety and risks, and how is it done in the law, and what are the rules for certifying AI risks, and how does that help with AI safety and risks, and what are the rules for certifying AI risks in the law, and how do they do it, and what are the rules for certifying AI risks, and how does that help AI safety and risks, and what does certification mean for AI risks and safety?","Based on the provided context from the AI Regulation Act, certification and monitoring of AI risks are conducted through a risk-based regulatory framework that classifies AI systems into categories such as high-risk and moderate-risk based on their potential impact on safety, rights, and national interest.\n\nFor high-risk AI systems, the law mandates mandatory algorithmic impact assessments, data privacy reviews, and sustainability screening before these systems can be certified for use. This certification process ensures that AI systems meet certain safety and ethical standards before deployment, directly contributing to managing and mitigating AI-related risks.\n\nThe rules for certifying AI risks involve:\n\n- Classifying AI systems by risk level (high-risk, moderate-risk, low-risk) depending on their potential effects.\n- Requiring high-risk AI systems to undergo comprehensive assessments (algorithmic impact, privacy, sustainability) prior to certification.\n

Semantic Chunking

In [68]:
from langchain_experimental.text_splitter import SemanticChunker

semantic_chunker = SemanticChunker(
    embeddings,
    breakpoint_threshold_type="percentile"
)

In [69]:
semantic_documents = semantic_chunker.split_documents(rag_documents[:20])

In [70]:
semantic_vectorstore = Qdrant.from_documents(
    semantic_documents,
    embeddings,
    location=":memory:",
    collection_name="AI_Bills_Data_Semantic_Chunks"
)

In [71]:
semantic_retriever = semantic_vectorstore.as_retriever(search_kwargs={"k" : 10})

In [72]:
semantic_retrieval_chain = (
    {"context": itemgetter("question") | semantic_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | rag_prompt | llm | StrOutputParser()
)

In [73]:
evaluate(
    semantic_retrieval_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "chain_init_semantic_chunk"},
)


View the evaluation results for experiment: 'clear-fire-83' at:
https://smith.langchain.com/o/73c7812f-fb16-4468-bb8f-115ba901532a/datasets/21fd1cce-754b-4695-81f3-4f8aefa58a02/compare?selectedSessions=7e17bc7d-b3d3-4e83-b5bc-28706a7e2341




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,How does the DOST relate to the NAIC's jurisdiction over AI development and regulation?,I don't know.,None,"The NAIC, which is attached to the Department of Science and Technology (DOST), has exclusive jurisdiction over all matters pertaining to AI, including its development, promotion, registration, and regulation. The NAIC is responsible for technical support and policy alignment across government agencies involved in AI, and it can impose administrative penalties for violations, establishing a direct relationship between DOST and the NAIC's AI oversight functions.",0,0,0,1.305145,5b0bdfed-5b90-4119-82a9-a30f2ac69a3f,a0605be9-9a7a-49be-b869-bcc54c23a600
1,"How does the development of Artificial Superintelligence (ASI) in the Philippines, as discussed in the context, relate to the global concerns about ASI surpassing human control and the need for responsible regulation to ensure safety and ethical use?","The development of Artificial Superintelligence (ASI) in the Philippines, as discussed in the context, acknowledges the global concerns about ASI surpassing human intelligence and potentially going beyond human control. The context highlights that there are risks associated with improperly secured or regulated ASI systems, which could lead to harmful consequences. Therefore, the Act aims to regulate the development and use of all types of AI, including ASI, to ensure safety, ethical use, and to prevent crimes, rights abuses, and unintended harms. It emphasizes the State’s responsibility to promote responsible and lawful AI innovation while protecting citizens and supporting national development. This reflects a commitment to addressing the global alarms around ASI by integrating responsible regulation and safety measures in the Philippine national framework.",None,"The context highlights that AI presents significant opportunities for the Philippines, including improving public services and advancing technological development. However, it also raises concerns about the rise of Artificial Superintelligence (ASI), a hypothetical AI system surpassing human intelligence and potentially beyond human control. Globally, scientific and policy communities have expressed alarms regarding the risks of ASI gaining unauthorized access to critical infrastructure and posing existential threats. The Philippines' approach, as outlined, emphasizes responsible development and regulation of AI, including ASI, to balance innovation with safety, ethics, and human oversight. This aligns with international concerns by advocating for policies that ensure ASI remains secure, transparent, and under meaningful human control to prevent misuse or unintended consequences.",1,1,0,5.271500,cec2f7f8-bf02-482c-b1d6-484da9ee5a5f,45b56bfa-af73-4341-ab84-1b377acddc17
2,"how do they do certification and monitoring of AI risks like in the law, and how does that relate to certification, and what does it mean for AI safety and risks, and how is it done in the law, and what are the rules for certifying AI risks, and how does that help with AI safety and risks, and what are the rules for certifying AI risks in the law, and how do they do it, and what are the rules for certifying AI risks, and how does that help AI safety and risks, and what does certification mean for AI risks and safety?",I don't know.,None,"The law states that the NAIC shall certify and monitor AI-related risks of all AI applications, which means they will evaluate and oversee AI systems to ensure safety and compliance. This certification process is part of a risk-based regulatory framework that classifies AI systems as high, moderate, or low risk based on their potential impact. By implementing certification and monitoring, the law aims to promote AI safety and manage risks effectively, ensuring AI systems align with ethical standards and social goals.",0,0